# chfem validation -- DRP Benchmarks

In [1]:
'''
ENVIRONMENT SETUP
'''

# install chfem
!pip install git+https://gitlab.com/cortezpedro/chfem_gpu.git@dev -q

# To enable c stdout printing in colab notebook
!pip install wurlitzer -q
import wurlitzer
wurlitzer.Wurlitzer.flush_interval=0.001
%load_ext wurlitzer

# necessary for pyvista plots
!pip install pyvista -q
!pip install piglet -q
!pip install pyvirtualdisplay -q
!apt-get -qq install xvfb
from pyvirtualdisplay import Display
display = Display(visible=0, size=(600, 400))
display.start()
def pv_plot3D(img, flip_z=False, cmap='Greys', opacity=None, label='voxels', outline=True, scalar_bar_args=None):
  grid = pv.ImageData()
  xyz = ( img.shape[2], img.shape[1], img.shape[0] )
  grid.dimensions = xyz
  grid.point_data[label] = np.reshape( img, (img.size) )
  if not flip_z:
    grid.point_data[label] = np.reshape( img, (img.size) )
  else:
    grid.point_data[label] = np.reshape( img[-1::-1,-1::-1,:], (img.size) )
  plotter = pv.Plotter(notebook=True)
  if opacity is None:
    plotter.add_volume(grid, scalars=label, cmap=cmap, scalar_bar_args=scalar_bar_args)
  else:
    plotter.add_volume(grid, scalars=label, cmap=cmap, opacity=opacity, scalar_bar_args=scalar_bar_args)
  if outline:
    plotter.add_mesh(grid.outline(), color='k')
  ax = plotter.add_axes(interactive=True)
  if flip_z:
    plotter.camera_position = [(xyz[0]*2.8, -xyz[1]*1.8, -xyz[0]*1.2),
                               (xyz[0]*0.8, xyz[1]*0.4, xyz[2]*0.4),
                               (-0.29,0.28,-0.9144)]
  plotter.show()
  return None

# other imports
from matplotlib import pyplot as plt
import pyvista as pv
import numpy as np
import chfem
import os
import urllib.request
import gzip

In [2]:
'''
INPUT ARGUMENTS FOR SIMULATIONS

ATTENTION: The following analyses are performed on supplementary material to
a digital petrophysics benchmarks paper. For more information, see:

(Andrä et al, 2013)
https://doi.org/10.1016/j.cageo.2012.09.005
https://doi.org/10.1016/j.cageo.2012.09.008

(Repository)
https://github.com/fkrzikalla/drp-benchmarks
'''

# Input arguments
#########################################################################################################
SAMPLE = 'fontainebleau'
#SAMPLE = 'berea'
#SAMPLE = 'grosmont'
#SAMPLE = 'spherepack'

SOLVER    = 'minres'
#SOLVER    = 'minres3'
#SOLVER    = 'minres2'
TOLERANCE = 1e-04
MAXIT     = 10_000

VIEW_SAMPLE_FLAG = True
VIEW_FIELDS_FLAG = (SOLVER != 'minres2')
#########################################################################################################

# Sample look-up table
#########################################################################################################
SAMPLE_TABLE = {
  'fontainebleau': {'raw': 'segmented-exxon.raw.gz',       'voxel_size': 7.50e-06, 'shape': ( 300, 288, 288), 'preprocess_image': None},
  'berea':         {'raw': 'segmented-vsg.raw.gz',         'voxel_size': 0.74e-06, 'shape': (1024,1024,1024), 'preprocess_image': lambda x: x[:,150:-150,150:-150]},
  'grosmont':      {'raw': 'segmented-vsg.raw.gz',         'voxel_size': 2.02e-06, 'shape': (1024,1024,1024), 'preprocess_image': lambda x: x[:512]},
  'spherepack':    {'raw': 'segmented-788x791x793.raw.gz', 'voxel_size': 7.00e-06, 'shape': ( 793, 791, 788), 'preprocess_image': lambda x: x[:550]}
}

if SAMPLE not in SAMPLE_TABLE:
  raise ValueError(f'Invalid sample name: {SAMPLE}\nPossible values: {SAMPLE_TABLE.keys()}')

SAMPLE_DATA = SAMPLE_TABLE[ SAMPLE ]

SHAPE  = SAMPLE_DATA['shape']
VOXEL  = SAMPLE_DATA['voxel_size']
RAW_GZ = SAMPLE_DATA['raw']

URL = f'https://raw.githubusercontent.com/fkrzikalla/drp-benchmarks/master/images/{SAMPLE}/{RAW_GZ}'
#########################################################################################################

In [3]:
'''
DOWNLOAD, EXTRACT, AND VISUALIZE SAMPLE
'''

# Get benchmark image
#########################################################################################################
cwd = os.getcwd()
filename = f'{cwd}/chfem_samples/{SAMPLE}_{RAW_GZ}'

# Check if samples directory exists
if not os.path.exists(f'{cwd}/chfem_samples'):
  os.makedirs(f'{cwd}/chfem_samples')

# Check if file has already been downloaded:
if not os.path.isfile(filename):
  print(f'Downloading image from {URL} to {filename}...')
  urllib.request.urlretrieve(URL,filename)
  print('Done')

print(f'Extracting image from {filename} ...')
with gzip.open(filename, 'rb') as f:
  raw_data = f.read()
print('Done')

img = np.frombuffer(raw_data, dtype='u1').reshape(SHAPE)

if SAMPLE_DATA['preprocess_image'] is not None:
  img = SAMPLE_DATA['preprocess_image'](img)
  SHAPE = img.shape

if VIEW_SAMPLE_FLAG:
  print('Rendering image ...')
  pore_color = 0
  solid_color = np.min(img[img>pore_color])
  my_cmap = lambda x: np.array( [ (0.,0.5,0.8,1.0) if d < solid_color/255 else (0.7,0.7,0.7,1.0) for d in x ]  )
  pv_plot3D( img, flip_z=True, cmap=my_cmap, opacity=1, label=f'voxels ({SAMPLE})')
  print('Done')
#########################################################################################################

In [4]:
'''
ABSOLUTE PERMEABILITY HOMOGENIZATION WITH chfem
'''

# Run simulations
#########################################################################################################
out_files = None
if VIEW_FIELDS_FLAG:
  out_files = f'{cwd}/chfem_samples/{SAMPLE}'

Keff = chfem.compute_property( 'permeability', img, voxel_size=VOXEL, direction='x',
                               solver=SOLVER, solver_tolerance=TOLERANCE, solver_maxiter=MAXIT,
                               output_fields=out_files )

Keff = np.array( Keff ) * ( 1e15 / 0.9869233 ) # conversion from m^2 to mD
print('Permeability tensor, in milliDarcy [mD]:')
print(Keff)

if VIEW_FIELDS_FLAG:
  print('Rendering velocity magnitude field ...')
  v = np.linalg.norm( chfem.import_vector_field_from_chfem(f'{out_files}_velocity_0.bin',SHAPE), axis=0 )
  v *= VOXEL**2 # quadratic scaling by VOXEL size. domain of simulations is normalized
  mu = np.mean(v)
  sigma = np.std(v)
  mask = v > mu+2*sigma
  v[np.logical_not(mask)] = 0.
  pv_plot3D(v, flip_z=True, cmap='jet', opacity='linear', label='velocity magnitude [m/s]')
  print('Done')
#########################################################################################################